In [1]:
import pandas as pd
import numpy as np
import random
import os
import json
from omegaconf import OmegaConf

In [23]:
conf = OmegaConf.load('../config.yaml')

In [24]:
conf

{'EDA': {'staging_data_folder': 'tft_houses/', 'clean_data_folder': 'dataframe_model/', 'df_information_houses': 'informations_households.csv', 'df_uk_holidays': 'uk_bank_holidays.csv', 'df_weather': 'weather_hourly_darksky.csv'}}

In [30]:
def read_house_csv(csv_file):
    df = pd.read_csv("../EDA/" + conf["EDA"]["staging_data_folder"] + csv_file)
    return df 

In [72]:
teste = read_house_csv("establishedAffluence_block_10_MAC001221.csv")
print(teste.head())

   Unnamed: 0      LCLid                         tstp energy(kWh/hh)
0      309822  MAC001221  2012-05-14 08:00:00.0000000          0.34 
1      309823  MAC001221  2012-05-14 08:30:00.0000000         0.321 
2      309824  MAC001221  2012-05-14 09:00:00.0000000         1.041 
3      309825  MAC001221  2012-05-14 09:30:00.0000000         0.345 
4      309826  MAC001221  2012-05-14 10:00:00.0000000         0.311 


In [86]:
teste[teste["energy(kWh/hh)"].isin(["Null"])]

,Unnamed: 0,LCLid,tstp,energy(kWh/hh)


In [81]:
teste[teste["energy(kWh/hh)"].isin(["Null"])]["energy(kWh/hh)"].index

Index([10431], dtype='int64')

In [85]:
df.iloc[df[df["energy(kWh/hh)"].isin(["Null"])]["energy(kWh/hh)"].index,3] = '0'

In [27]:
conf["EDA"]["df_uk_holidays"]

'uk_bank_holidays.csv'

In [91]:
def get_household(csv_file):
    def read_house_csv(csv_file):
        df = pd.read_csv("../EDA/" + conf["EDA"]["staging_data_folder"] + csv_file)
        return df 

    holidays = pd.read_csv("../EDA/"+ conf["EDA"]["df_uk_holidays"])
    holidays['Bank holidays'] = pd.to_datetime(holidays['Bank holidays'])
    holidays = holidays.loc[(holidays['Bank holidays'].dt.year >= 2013)]
    df = read_house_csv(csv_file)
    df = df.rename(columns={'LCLid':'house_hold','tstp':'time','energy(kWh/hh)':'Energy_kwh'})
    df.iloc[df[df["Energy_kwh"].isin(["Null"])]["Energy_kwh"].index,3] = '0'
    df.Energy_kwh = pd.to_numeric(df.Energy_kwh)
    df.time = pd.to_datetime(df.time)
    df = df.loc[((df['time'].dt.year >= 2013) & (df.time.dt.month.isin([1,2,3,4,5,6,7,8,9,10,11,12]))) | (df['time'].dt.year == 2014)]
    df['holiday'] = df['time'].isin(holidays['Bank holidays']) #(df.time.dt.day_of_week >= 5) | (df['time'].isin(holidays['Bank holidays']))
    df['holiday'] = df['holiday'].astype(int)
    #df = df[df.house_hold == house_selected]
    
    return df

In [92]:
df_teste = get_household("establishedAffluence_block_10_MAC001221.csv")

In [93]:
def transform_half_in_hourly(df : pd.DataFrame):
    """ 
        Description : a function where transform a dataframe with a halfhourly datetime in a hourly datetime dataframe
        Input : df : pd.Dataframe
        return : df : pd.Dataframe
    """
    list_df = []
    for house in df.house_hold.unique():
        df_temp = df[df['house_hold'] == house]
        df_temp = df[['time','Energy_kwh']]
        df_temp.set_index('time',inplace=True)
        df_temp = df_temp.resample('h').sum()
        df_temp['house_hold'] = house
        df_temp = df_temp.reset_index()
        list_df.append(df_temp)
    df_final = pd.concat(list_df, ignore_index=True)
    print(df_final.shape)
    return df_final

transform_half_in_hourly(df_teste)

(10153, 3)


,time,Energy_kwh,house_hold
0,2013-01-01 00:00:00,1.177,MAC001221
1,2013-01-01 01:00:00,0.756,MAC001221
2,2013-01-01 02:00:00,0.328,MAC001221
3,2013-01-01 03:00:00,0.403,MAC001221
4,2013-01-01 04:00:00,0.561,MAC001221
...,...,...,...
10148,2014-02-27 20:00:00,1.097,MAC001221
10149,2014-02-27 21:00:00,0.654,MAC001221
10150,2014-02-27 22:00:00,0.647,MAC001221
10151,2014-02-27 23:00:00,0.497,MAC001221


In [ ]:

def get_household(house):
    block = house.split('_')[1] + '_' + house.split('_')[2]
    house_selected = house.split('_')[-1]
    holidays = pd.read_csv('uk_bank_holidays.csv')
    holidays['Bank holidays'] = pd.to_datetime(holidays['Bank holidays'])
    holidays = holidays.loc[(holidays['Bank holidays'].dt.year >= 2013)]
    df = pd.read_csv(f'halfhourly_dataset/halfhourly_dataset/{block}.csv')
    df = df.rename(columns={'LCLid':'house_hold','tstp':'time','energy(kWh/hh)':'Energy_kwh'})
    df.iloc[df[df.Energy_kwh == 'Null'].Energy_kwh.index,2] = '0'
    df.Energy_kwh = pd.to_numeric(df.Energy_kwh)
    df.time = pd.to_datetime(df.time)
    df = df.loc[((df['time'].dt.year >= 2013) & (df.time.dt.month.isin([1,2,3,4,5,6,7,8,9,10,11,12]))) | (df['time'].dt.year == 2014)]
    df['holiday'] = df['time'].isin(holidays['Bank holidays']) #(df.time.dt.day_of_week >= 5) | (df['time'].isin(holidays['Bank holidays']))
    df['holiday'] = df['holiday'].astype(int)
    #df = df[df.house_hold == house_selected]
    print(df.head())
    
    return df, house

def transform_half_in_hourly(df : pd.DataFrame):
    """ 
        Description : a function where transform a dataframe with a halfhourly datetime in a hourly datetime dataframe
        Input : df : pd.Dataframe
        return : df : pd.Dataframe
    """
    list_df = []
    for house in df.house_hold.unique():
        df_temp = df[df['house_hold'] == house]
        df_temp = df[['time','Energy_kwh']]
        df_temp.set_index('time',inplace=True)
        df_temp = df_temp.resample('h').sum()
        df_temp['house_hold'] = house
        df_temp = df_temp.reset_index()
        list_df.append(df_temp)
    df_final = pd.concat(list_df, ignore_index=True)
    print(df_final.shape)
    return df_final


def add_weater_data(df:pd.DataFrame,df_weather:pd.DataFrame):
    df_merged = pd.merge(df,df_weather,how='left',on = 'time')
    print(df_merged.head())
    return df_merged

def add_holidays(df:pd.DataFrame):
    holidays = pd.read_csv('uk_bank_holidays.csv')
    holidays['Bank holidays'] = pd.to_datetime(holidays['Bank holidays'])
    holidays = holidays.loc[(holidays['Bank holidays'].dt.year >= 2013)]
    df['holiday'] = df['time'].isin(holidays['Bank holidays']) #(df.time.dt.day_of_week >= 5) | (df['time'].isin(holidays['Bank holidays']))
    df['holiday'] = df['holiday'].astype(int)
    return df

def add_bool_weather_missing_values(df:pd.DataFrame):
    df['bool_weather_missing_values'] = df['temperature'].apply(lambda x: 1 if x == np.nan else 0)
    return df

def dict_of_dict_labels(df:pd.DataFrame):
    """a function where get all categorical columns in a df and save ache categorie as a number in a dict, for the end all dicts is added in a final dict
    """
    dict_of_dict_labels = {}
    categorical_cols = df.select_dtypes(include='object').columns
    for column in categorical_cols:
        dict_label = {}
        catogories = df[column].unique()
        for i in range(len(catogories)):
            dict_label[catogories[i]] = i
        dict_of_dict_labels[column] = dict_label
    
    return dict_of_dict_labels

def labeling_categorical_features(df:pd.DataFrame):
    dict_labels = dict_of_dict_labels(df)
    
    for key,value in dict_labels.items(): #value is a dict
        label_dict = {k: v for k, v in value.items()}
        df[key] = df[key].map(label_dict).astype(str)
    
    return df, dict_labels


#if add a diferent feature in the future change this function
def feature_eng_function(df:pd.DataFrame):
    #df['year'] = df['time'].dt.year.astype(str)
    #df['month'] = df['time'].dt.month.astype(str)
    #df['day'] = df['time'].dt.day.astype(str)
    #df['hour'] = df['time'].dt.hour.astype(str)
    #df['dayofweek_num'] = df['time'].dt.weekday.astype(str)
    df['temperature'] = df['temperature'].apply(lambda x: 0 if x == np.nan else x)
    df['windSpeed'] = df['windSpeed'].apply(lambda x: 0 if x == np.nan else x)
    df, dict_labels = labeling_categorical_features(df)
    return df, dict_labels


def save_json(dict_labels,house_selcted):
    file_path = f'{house_selcted}.json'  

    with open(file_path, "w") as f:
        json.dump(dict_labels, f)
        
    source_path = f'{house_selcted}.json'
    destination_path = f'json_files\{house_selcted}.json'

    # Check if source file exists
    if os.path.isfile(source_path):
        try:
            # Move the file using os.rename()
            os.rename(source_path, destination_path)
            print(f"File moved successfully from {source_path} to {destination_path}")
        except OSError as e:
            # Handle potential errors during move
            if "Destination exists" in str(e):  # Check for specific error
                print(f"Error: A file already exists at {destination_path}.")
            else:
                print(f"An error occurred during the move: {e}")
    else:
        print(f"Error: Source file '{source_path}' not found.")

In [98]:
opa = pd.read_csv("../EDA/dataframe_model/stretchedSociety_block_109_MAC005547.csv")

opa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10153 entries, 0 to 10152
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   time                         10153 non-null  object 
 1   Energy_kwh                   10153 non-null  float64
 2   house_hold                   10153 non-null  int64  
 3   temperature                  10151 non-null  float64
 4   windSpeed                    10151 non-null  float64
 5   precipType                   10153 non-null  int64  
 6   icon                         10153 non-null  int64  
 7   summary                      10153 non-null  int64  
 8   holiday                      10153 non-null  int64  
 9   bool_weather_missing_values  10153 non-null  int64  
 10  year                         10153 non-null  int64  
 11  month                        10153 non-null  int64  
 12  day                          10153 non-null  int64  
 13  hour            